# AIMO Prize 3 - OOP Baseline Submission (v5)

This notebook demonstrates how to use the Object-Oriented Baseline with **Few-Shot Prompting**, **Majority Voting**, and **Parquet Submission**.

## Setup Instructions
1. **Add Utility Script**: Upload `src/kaggle_baseline.py` as a Dataset (e.g., named `aimo-pp3-modules`).
2. **Add Model**: Search for and add the model `Qwen/Qwen2.5-Math-7B-Instruct`.
3. **Attach Competition Data**: Ensure the AIMO 3 competition data is attached.
4. **Run**: Execute the cells below.

In [ ]:
import os
import glob
import sys

print('--- DIAGNOSTIC INFO ---')
print(f'Current working directory: {os.getcwd()}')
print('Contents of /kaggle/input/:')
try:
    for item in sorted(os.listdir('/kaggle/input')):
        full_path = os.path.join('/kaggle/input', item)
        if os.path.isdir(full_path):
            print(f'  {item}/')
            if any(x in item.lower() for x in ['competition', 'prize', 'module', 'qwen']):
                try:
                    for sub_item in sorted(os.listdir(full_path)):
                        print(f'    - {sub_item}')
                except Exception as e:
                    print(f'    (Error listing {item}: {e})')
        else:
            print(f'  {item}')
except Exception as e:
    print(f'Error listing /kaggle/input/: {e}')

print('\nEnvironment variables:')
for key, value in sorted(os.environ.items()):
    if key.startswith('KAGGLE') or 'PATH' in key or 'HOME' in key:
        print(f'{key}={value}')

print('--- END DIAGNOSTIC INFO ---')


In [ ]:
import sys
import os
import glob

# 1. Import the OOP Baseline Module
sys.path.append('/kaggle/input/aimo-pp3-modules')
sys.path.append(os.path.join(os.getcwd(), 'src')) 

try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM, LocalVLLM
    print("✅ Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"❌ Import Failed: {e}. Make sure the 'aimo-pp3-modules' dataset is attached.")

In [ ]:
# 2. Configure Environment & Model
config = CompetitionConfig()
config.n_repetitions = 1 

print(f"Environment: {'Kaggle' if config.is_kaggle else 'Local'}")

if config.is_kaggle and os.path.exists(config.model_path):
    # Uncomment for Real Model (vLLM)
    # llm = LocalVLLM(config.model_path)
    print("⚠️ Using MockLLM. Uncomment LocalVLLM lines to use the real model.")
    llm = MockLLM()
else:
    llm = MockLLM()

solver = AIMSolver(config, llm)

## Submission Loop (Parquet)

In [ ]:
import sys
import os
import glob
import pandas as pd

# --------------------------------------------------------------------------------
# 3. AIMO 3 API Setup (Inference Server Pattern)
# --------------------------------------------------------------------------------
aimo_server_mod = None

# Locate the API files
api_files = glob.glob("/kaggle/input/**/aimo_3_inference_server.py", recursive=True)
if not api_files:
    api_files = glob.glob("data/**/aimo_3_inference_server.py", recursive=True)

test_csv_path_gateway = '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv'
print(f'[API Setup] Checking for Gateway test.csv: {os.path.exists(test_csv_path_gateway)}')
if not os.path.exists(test_csv_path_gateway):
    print(f'[API Setup] test.csv NOT FOUND at {test_csv_path_gateway}')
    comp_dir = os.path.dirname(test_csv_path_gateway)
    if os.path.exists(comp_dir):
        print(f'[API Setup] Contents of {comp_dir}/: {os.listdir(comp_dir)}')
    else:
        print(f'[API Setup] Competition directory {comp_dir} does not exist.')

found_test_csv = glob.glob('/kaggle/input/**/test.csv', recursive=True)
if found_test_csv:
    print(f'[API Setup] Found test.csv at: {found_test_csv[0]}')
else:
    print('[API Setup] No test.csv found via glob.')
if api_files:
    api_path = os.path.dirname(api_files[0])
    # Add parent of 'kaggle_evaluation' to sys.path if needed
    if os.path.basename(api_path) == 'kaggle_evaluation':
        parent_dir = os.path.dirname(api_path)
        if parent_dir not in sys.path:
            sys.path.append(parent_dir)
    elif api_path not in sys.path:
        sys.path.append(api_path)

    try:
        # Try importing as a module (preferred)
        import aimo_3_inference_server as aimo_server_mod
        print("✅ Imported aimo_3_inference_server.")
    except ImportError:
        try:
            # Try importing from package
            from kaggle_evaluation import aimo_3_inference_server as aimo_server_mod
            print("✅ Imported aimo_3_inference_server from package.")
        except ImportError as e:
            print(f"❌ Failed to import API: {e}")

# Define the predict function
def predict(*args, **kwargs):
    try:
        # Args usually contain (problem_batch, id_batch)
        # We expect the first arg to contain the problem.
        input_data = args[0] if args else None
        problem_text = "What is 0+0?" # Default fallback

        # Logic to extract problem string from DataFrame (Polars/Pandas) or Series
        if hasattr(input_data, 'columns') and 'problem' in input_data.columns:
            # It's a DataFrame-like object
            val = input_data['problem'][0]
            problem_text = str(val)
        elif hasattr(input_data, 'name') and input_data.name == 'problem':
             # It's a Series
             val = input_data[0]
             problem_text = str(val)
        elif isinstance(input_data, str):
            problem_text = input_data
        
        # Solve
        answer = solver.solve(problem_text)
        return answer
    except Exception as e:
        print(f"Predict Error: {e}")
        return 0

# Start Server
if aimo_server_mod:
    print("Initializing Inference Server...")
    # The server expects endpoint listeners as arguments
    server = aimo_server_mod.AIMO3InferenceServer(predict)
    
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        print("Starting Server (Blocking) for Competition Rerun...")
        server.serve()
    else:
        print("Running Local Gateway for Validation/Save Version...")
        # This runs against the local test set (or Kaggle public test set)
        # and AUTOMATICALLY writes submission.parquet
        try:
            # Use the found test.csv path for local validation
            actual_test_csv = found_test_csv[0] if 'found_test_csv' in locals() and found_test_csv else '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv'
            server.run_local_gateway(data_paths=(actual_test_csv,))
            print("✅ Local Gateway finished.")
        except Exception as e:
            print(f"❌ Local Gateway failed: {e}")
            # Fallback: Create dummy file if gateway crashed (to allow 'Save Version' to pass)
            print("Creating fallback submission.parquet...")
            pd.DataFrame({'id': ['test_id'], 'answer': [0]}).to_parquet('submission.parquet', index=False)
else:
    print("⚠️ API not found. Creating dummy submission.")
    pd.DataFrame({'id': ['test_id'], 'answer': [0]}).to_parquet('submission.parquet', index=False)
